# Crawler e Monta a Base

In [ ]:
%%time

import tweepy
import time
import pandas as pd
import sqlite3
import numpy as np
pd.set_option('display.max_columns', 100)


TWITTER_BEARER_TOKEN="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

client = tweepy.Client(bearer_token = TWITTER_BEARER_TOKEN)

con = sqlite3.connect(r'base_twitter.db')
cur = con.cursor()

base_total = pd.read_sql("SELECT * FROM tweets", con = con)

for keyword in ['#bolsonaropresidente', '#LulaPresidente', 'bolsoladrao', 'luladrao', 'Lula', 'Bolsonaro']: 
    
    print(keyword)
    
    tweets_lista = []
    colunas = []
    
    id_usuario = [] 
    usuario = [] 
    usuario_tela = [] 
    data_criacao = [] 
    n_seguidores = [] 
    n_seguidos = [] 
    n_tweets = [] 
    perfil_fechado = [] 
    verificado = [] 
    tem_background_imagem = [] 
    
    tweets = client.search_recent_tweets(query=keyword, 
                                         tweet_fields=['context_annotations', 'created_at'],
                                         user_fields=['public_metrics', 'created_at', 'verified', 'profile_image_url', 
                                                      'protected'],
                                         expansions='author_id', max_results=100)

    users = {u["id"]: u for u in tweets.includes['users']}

    for tweet in tweets.data:
        if users[tweet.author_id]:
            user = users[tweet.author_id]
            id_usuario.append(user["id"])
            usuario.append(user["username"])
            usuario_tela.append(user["name"])
            data_criacao.append(str(user.created_at))
            n_seguidores.append(user.public_metrics['followers_count'])
            n_seguidos.append(user.public_metrics['following_count'])
            n_tweets.append(user.public_metrics['tweet_count'])
            perfil_fechado.append(user.protected)
            verificado.append(user.verified)
            tem_background_imagem.append(user.profile_image_url)
            
    df = pd.DataFrame({'ID': id_usuario,
                       'Usuario': usuario,
                       'Usuario_Tela': usuario_tela,
                       'Data_Criacao': data_criacao,
                       'N_Seguidores': n_seguidores,
                       'N_Seguidos': n_seguidos,
                       'N_Tweets': n_tweets,
                       'Verificado': verificado,
                       'Fechado': perfil_fechado,
                       'Background_Img': tem_background_imagem})
    
    df_tweets = pd.DataFrame()

    for i in df['ID']:

        tweets_lista = [str(i)]
        colunas = ['ID']

        j = 1

        for aux in tweepy.Paginator(client.get_users_tweets, i, max_results = 100, exclude = "retweets", limit = 2):
            for v in range(0, 100):
                colunas.append("tweet_" + str(j))
                j += 1
                try:
                    tweets_lista.append([aux[0][v]['text']])
                except:
                    tweets_lista.append(None)
        try:
            d = dict(zip(colunas, tweets_lista))
            df_aux = pd.DataFrame(data = d)
            df_tweets = df_tweets.append(df_aux, ignore_index = True)
        except:
            print(i)
        
        
    df['ID'] = df['ID'].astype(str)
    try:
        base = pd.merge(df, df_tweets, on = 'ID', how = 'left')
    except:
        base = df
        
    base['keyword'] = keyword
    
    base['troca_sent'] = 0
    base['sent_neg'] = 0
    base['sent_pos'] = 0
    base['sent_neu'] = 0

    base['total_vb'] = 0
    base['total_adj'] = 0
    base['total_adv'] = 0
    base['total_pron'] = 0
    base['total_hashtag'] = 0

    s = SentimentIntensityAnalyzer()

    for i in range(0, base.shape[0]):

        troca_sent = 0

        lista_troca_sent = ["0"]

        sent_neg = [0]
        sent_neu = [0] 
        sent_pos = [0]

        tamanho = 0
        total_vb = 0
        total_adj = 0
        total_adv = 0
        total_pron = 0
        total_hashtag = 0
        total_emoji = 0
        total_tk = 0

        k = 1

        for j in range(10, 210):
            aux = str(base.iloc[i, j])

            ###### analise sintatica
            if aux != None or aux != np.nan or aux != str(nan):
                tamanho = tamanho + 1

                lista = cont_tipo(aux)

                total_vb = total_vb + lista[0]
                total_adj = total_adj + lista[1]
                total_adv = total_adv + lista[2]
                total_pron = total_pron + lista[3]
                total_hashtag = total_hashtag + lista[4]

            ###### polaridade
                pol = s.polarity_scores(aux)

                list_pol = [pol['neg'], pol['neu'], pol['pos']]
                if list_pol.index(max(list_pol)) == 0:
                    sent_neg.append(1)
                    lista_troca_sent.append("neg")
                elif list_pol.index(max(list_pol)) == 1:
                    sent_neu.append(1)
                    lista_troca_sent.append("neu")
                elif list_pol.index(max(list_pol)) == 2:
                    sent_pos.append(1)
                    lista_troca_sent.append("pos")

                if lista_troca_sent[k] != lista_troca_sent[k - 1]:
                    troca_sent += 1

                k += 1
            else:
                print(j)
                
        base.loc[i, 'total_vb'] = total_vb
        base.loc[i, 'total_adj'] = total_adj
        base.loc[i, 'total_adv'] = total_adv
        base.loc[i, 'total_pron'] = total_pron
        base.loc[i, 'total_hashtag'] = total_hashtag
        base.loc[i, 'total_emoji'] = total_emoji
        base.loc[i, 'total_tk'] = total_tk

        base.loc[i, 'sent_neg'] = sum(sent_neg)
        base.loc[i, 'sent_neu'] = sum(sent_neu)
        base.loc[i, 'sent_pos'] = sum(sent_pos)

        base.loc[i, 'troca_sent'] = troca_sent
    
    base.drop_duplicates(['ID'], inplace = True)
    
    try:
        base_total.drop(['index'], 1, inplace = True)
    except:
        print(" ")

    base.to_sql("tweets", con = con, if_exists = 'append', index = False)
    
    time.sleep(15)
    
con.close()

con = sqlite3.connect(r'base_twitter.db')
cur = con.cursor()

base_total = pd.read_sql("SELECT * FROM tweets", con = con)
base_total['media_vb'] = [base_total.loc[i, 'total_vb']/base_total.loc[i, 'total_tk'] for i in range(0, base_total.shape[0])]
base_total['media_adj'] = [ base_total.loc[i, 'total_adj']/base_total.loc[i, 'total_tk'] for i in range(0, base_total.shape[0])]
base_total['media_adv'] = [ base_total.loc[i, 'total_adv']/base_total.loc[i, 'total_tk'] for i in range(0, base_total.shape[0])]
base_total['media_pron'] = [ base_total.loc[i, 'total_pron']/base_total.loc[i, 'total_tk'] for i in range(0, base_total.shape[0])]
base_total['media_hashtag'] = [ base_total.loc[i, 'total_hashtag']/base_total.loc[i, 'total_tk'] for i in range(0, base_total.shape[0])]
base_total['media_emoji'] = [ base_total.loc[i, 'total_emoji']/base_total.loc[i, 'total_tk'] for i in range(0, base_total.shape[0])]
base_total['Ano_Criacao'] = pd.DatetimeIndex(base_total.Data_Criacao).year
base_total['Tem_Background_Img'] = [1 if base_total.loc[i, 'Background_Img'] else 0 for i in range(0, base_total.shape[0])]
base_total.drop_duplicates(['ID'], inplace = True)
base_total.to_sql("tweets", con = con, if_exists="replace", index = False)

con.close()

#bolsonaropresidente


<timed exec>:189: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


 
#LulaPresidente
1098268451494342658
1098268451494342658
 
bolsoladrao
 
luladrao
1103687320350412800
1125022835175514113
 
Lula
1500265842654822401
1057720765737504768
1266957929468608514
1057720765737504768
2815705718
178912987
 
Bolsonaro
1123055030985273344
1156751260336500736
1123055030985273344
53900405
1149100684014510080
